In [1]:
import os
import cv2
import random
import math
import numpy as np
import datetime as dt
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/Users/vedantpadole/Desktop/Research/Adjectives_real')

all_classes = os.listdir('/Users/vedantpadole/Desktop/Research/Adjectives_real')
print (all_classes)  
len(all_classes)

['Sick', 'Dry', 'Healthy']


3

In [21]:
import os

folder_path = "/Users/vedantpadole/Desktop/Research/Adjectives"
sub_folder_name = "/Users/vedantpadole/Desktop/Research/Adjectives/.DS_Store"

sub_folder_path = os.path.join(folder_path, sub_folder_name)

os.remove(sub_folder_path)


In [22]:
dataset_path = os.listdir('/Users/vedantpadole/Desktop/Research/Adjectives')

all_classes = os.listdir('/Users/vedantpadole/Desktop/Research/Adjectives')
print (all_classes)  
len(all_classes)

['Sick', 'Dry', 'Healthy']


3

In [4]:
IMAGE_HEIGHT,IMAGE_WIDTH=128,128
SEQUENCE_LENGTH=10
DATASET='/Users/vedantpadole/Desktop/Research/Adjectives_real'
CLASSES_LIST=all_classes

In [5]:
def frames_extraction(video_path):
  frames_list=[]
  video_reader=cv2.VideoCapture(video_path)
  video_frames_count=int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
  skip_frames_window=max(int(video_frames_count/SEQUENCE_LENGTH),1)
  for frame_counter in range(SEQUENCE_LENGTH):
    video_reader.set(cv2.CAP_PROP_POS_FRAMES,frame_counter*skip_frames_window)
    success,frame=video_reader.read()
    if not success:
      break
    resized_frame=cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))
    normalized_frame=resized_frame/255
    frames_list.append(normalized_frame)
  video_reader.release()
  return frames_list

In [6]:
def create_dataset():
  features=[]
  labels=[]
  video_files_path=[]
  for class_index,class_name in enumerate(CLASSES_LIST):
    print(f'Extracting the data of class:  {class_name}')
    files_list=os.listdir(os.path.join(DATASET,class_name))
    for file_name in files_list:
      video_file_path=os.path.join(DATASET,class_name,file_name)
      frames=frames_extraction(video_file_path)
      if len(frames)==SEQUENCE_LENGTH:
        features.append(frames)
        labels.append(class_index)
        video_files_path.append(video_file_path)
  features=np.asarray(features)
  labels=np.array(labels)
  return features,labels,video_files_path

In [7]:
features,labels,video_files_path=create_dataset()

Extracting the data of class:  Sick
Extracting the data of class:  Dry
Extracting the data of class:  Healthy


In [8]:
from keras.utils import to_categorical
one_hot_encoded_labels=to_categorical(labels)

In [9]:
seed_constant=27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [10]:
from sklearn.model_selection import train_test_split
features_train,features_test,labels_train,labels_test=train_test_split(features,one_hot_encoded_labels,test_size=0.2,shuffle=True,random_state=seed_constant)

In [11]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,TimeDistributed,Dropout,Flatten,Dense,LSTM
model=Sequential()
model.add(TimeDistributed(Conv2D(16,(3,3),padding='same',activation='relu'),input_shape=(SEQUENCE_LENGTH,IMAGE_HEIGHT,IMAGE_WIDTH,3)))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Conv2D(32,(3,3),padding='same',activation='relu')))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Conv2D(64,(3,3),padding='same',activation='relu')))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Conv2D(128,(3,3),padding='same',activation='relu')))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64))
model.add(Dense(len(CLASSES_LIST),activation='softmax'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 10, 128, 128, 16)  448      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 10, 64, 64, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 10, 64, 64, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 10, 64, 64, 32)   4640      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 10, 32, 32, 32)   0

In [13]:
from keras.callbacks import EarlyStopping
early_stopping_call=EarlyStopping(monitor='val_loss',patience=10,mode='min',restore_best_weights=True)
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
r=model.fit(x=features_train,y=labels_train,epochs=50,batch_size=32,shuffle=True,validation_split=0.2,callbacks=[early_stopping_call])

Epoch 1/50
7/7 [==============================] - 26s 3s/step - loss: 1.5605 - accuracy: 0.3532 - val_loss: 1.1213 - val_accuracy: 0.2941
Epoch 2/50
7/7 [==============================] - 22s 3s/step - loss: 1.0985 - accuracy: 0.3532 - val_loss: 1.0977 - val_accuracy: 0.3529
Epoch 3/50
7/7 [==============================] - 25s 4s/step - loss: 1.0986 - accuracy: 0.3682 - val_loss: 1.0993 - val_accuracy: 0.3529
Epoch 4/50
7/7 [==============================] - 27s 4s/step - loss: 1.0982 - accuracy: 0.3682 - val_loss: 1.1063 - val_accuracy: 0.3529
Epoch 5/50
7/7 [==============================] - 28s 4s/step - loss: 1.1011 - accuracy: 0.3682 - val_loss: 1.1061 - val_accuracy: 0.3529
Epoch 6/50
7/7 [==============================] - 28s 4s/step - loss: 1.1031 - accuracy: 0.3682 - val_loss: 1.1043 - val_accuracy: 0.3529
Epoch 7/50
7/7 [==============================] - 28s 4s/step - loss: 1.0985 - accuracy: 0.3682 - val_loss: 1.1071 - val_accuracy: 0.3529
Epoch 8/50
7/7 [==================

In [14]:
model.save("Indian_sign_language.h5")